In [45]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [46]:
def to_utc(date):
    #This function converts an object to UTC. 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-06-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-06-30", "%Y-%m-%d")

# get data from June 1 to June 30
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields = "author", "subreddit", "created_utc", "num_comments","score","title","selftext"
subreddit = 'leaguepbe'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        #added this part from the module  
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-06-01 to 2020-06-02
=====Done
Doing 2020-06-02 to 2020-06-03
=====Done
Doing 2020-06-03 to 2020-06-04
=====Done
Doing 2020-06-04 to 2020-06-05
=====Done
Doing 2020-06-05 to 2020-06-06
=====Done
Doing 2020-06-06 to 2020-06-07
=====Done
Doing 2020-06-07 to 2020-06-08
=====Done
Doing 2020-06-08 to 2020-06-09
=====Done
Doing 2020-06-09 to 2020-06-10
=====Done
Doing 2020-06-10 to 2020-06-11
=====Done
Doing 2020-06-11 to 2020-06-12
=====Done
Doing 2020-06-12 to 2020-06-13
=====Done
Doing 2020-06-13 to 2020-06-14
=====Done
Doing 2020-06-14 to 2020-06-15
=====Done
Doing 2020-06-15 to 2020-06-16
=====Done
Doing 2020-06-16 to 2020-06-17
=====Done
Doing 2020-06-17 to 2020-06-18
=====Done
Doing 2020-06-18 to 2020-06-19
=====Done
Doing 2020-06-19 to 2020-06-20
=====Done
Doing 2020-06-20 to 2020-06-21
=====Done
Doing 2020-06-21 to 2020-06-22
=====Done
Doing 2020-06-22 to 2020-06-23
=====Done
Doing 2020-06-23 to 2020-06-24
=====Done
Doing 2020-06-24 to 2020-06-25
=====Done
Doing 2020-06-25

In [47]:
results

[[{'author': 'BarisberatWNR',
   'created_utc': 1591027621,
   'num_comments': 3,
   'score': 1,
   'selftext': 'Description:  \nIf you want to screenshot using F12 key, you cant bacause if you do that, the game closes.\n\nSteps:  \n1. Go to a random game  \n2. Take a screenshot using F12  \n3. Instead of screenshotting the game and putting on the "Screenshots" folder inside your League of Legends (PBE) folder, the game instead, just closes',
   'subreddit': 'LeaguePBE',
   'title': '[Game] - Screenshotting closes the game'}],
 [{'author': 'MuffinLoL',
   'created_utc': 1591138026,
   'num_comments': 2,
   'score': 2,
   'selftext': 'So this bug is pretty old one since it\'s been there for few days already(and some of it got reported as well).\n\nBasically:\n\n- None of the rewards from TFT Galaxies Pass II+ can be obtained\n\n- All rewards are unlocked in the Pass page, but it says "locked" when you wanna use them\n\n- Some players reported they were granted(?) the rewards this mornin

In [48]:
flat_list = []
#loop through the reddit results
for sublist in results: #ve empty lists is because there are days wherein there are no submissions
    if sublist is not None:
    #check if sublist is not empty. The reason we ha
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head(10))
df.to_csv("reddit_lol.csv")

,author,created_utc,num_comments,score,selftext,subreddit,title
0,BarisberatWNR,1591027621,3,1,Description: \nIf you want to screenshot usin...,LeaguePBE,[Game] - Screenshotting closes the game
1,MuffinLoL,1591138026,2,2,So this bug is pretty old one since it's been ...,LeaguePBE,Galaxies Pass II+ Rewards bug
2,Catman_PBE,1591395122,4,1,"""At approximately 12:00 PM PDT on Tuesday, 09 ...",LeaguePBE,"PBE Match History will be wiped on June 9th, 2..."
3,dKiWiKiD,1591737077,178,44,Hey All!\n\nhttps://imgur.com/wIKptIM\n\nPool ...,LeaguePBE,PBE Feedback And Bug Report Thread: Pool Party...
4,Papuchochoe,1591740720,10,16,Hello!\n\nSo I saw new Nurse Akali VFX and I r...,LeaguePBE,Nurse Akali updated VFX feedback/question
5,dKiWiKiD,1591737220,169,9,Hey All!\n\nhttps://imgur.com/P0HYxUt\nPool Pa...,LeaguePBE,PBE Feedback And Bug Report Thread: Pool Party...
6,dKiWiKiD,1591736986,54,4,Hey All!\n\nhttps://imgur.com/cI51E7Q\n\nPool ...,LeaguePBE,Pool Party Orianna
7,zheung14,1591715537,7,4,I can't find Hextech Nocturne in Loot now.\n\n...,LeaguePBE,I can't find Hextech Nocturne in Loot.
8,dKiWiKiD,1591737146,23,0,Hey All!\n\nhttps://imgur.com/oPKNdCB\nPool Pa...,LeaguePBE,PBE Feedback And Bug Report Thread: Pool Party...
9,dKiWiKiD,1591736823,17,0,Hey All!\n\nhttps://imgur.com/KineXCq\nPool Pa...,LeaguePBE,PBE Feedback And Bug Report Thread: Pool Party...


In [49]:
len(df.index) # Number of Submission

93

In [50]:
# remove author with "[deleted]" value since it skews the data and shows as the highest submissions
a= df[df.author !='[deleted]'] 
# count the number of submission by getting the frequency/ how many times the author appeared in the data. Then, return the name of the author with highest number of count
a.author.value_counts().idxmax()

'Kingby'

In [51]:
df.nlargest(1,['score'])[['author','score']] # displays the author with highest score

,author,score
10,Geobirdd,52


In [52]:
c = df[df.num_comments==df.num_comments.max()] # displays the record with highest number of comments
c[['author', 'num_comments']]

,author,num_comments
49,Kingby,284


In [53]:
#Converting epoch time to UTC
#Coverting UTC to readable date
df['created_utc']=pd.to_datetime(df['created_utc'], unit ='s')
df["created_utc"]= pd.to_datetime(df["created_utc"].dt.strftime("%m/%d/%y"))
display(df.head(5))

,author,created_utc,num_comments,score,selftext,subreddit,title
0,BarisberatWNR,2020-06-01,3,1,Description: \nIf you want to screenshot usin...,LeaguePBE,[Game] - Screenshotting closes the game
1,MuffinLoL,2020-06-02,2,2,So this bug is pretty old one since it's been ...,LeaguePBE,Galaxies Pass II+ Rewards bug
2,Catman_PBE,2020-06-05,4,1,"""At approximately 12:00 PM PDT on Tuesday, 09 ...",LeaguePBE,"PBE Match History will be wiped on June 9th, 2..."
3,dKiWiKiD,2020-06-09,178,44,Hey All!\n\nhttps://imgur.com/wIKptIM\n\nPool ...,LeaguePBE,PBE Feedback And Bug Report Thread: Pool Party...
4,Papuchochoe,2020-06-09,10,16,Hello!\n\nSo I saw new Nurse Akali VFX and I r...,LeaguePBE,Nurse Akali updated VFX feedback/question


In [54]:
import datetime as dt

In [55]:
print(df.created_utc.value_counts().index.max()) #Date with highest submission
print(df.created_utc.value_counts().index.max().strftime("%A")) #Day of the week with highest submission

2020-06-30 00:00:00
Tuesday
